# Init Bionic VTOL

In [1]:
include("../Flyonic.jl");
using .Flyonic;

using Rotations; # used for initial position

using ReinforcementLearning;
using StableRNGs;
using Flux;
using Flux.Losses;
using Random;
using IntervalSets;
using LinearAlgebra;
using Distributions;

using Plots;
using Statistics;

using BSON: @save, @load # save mode

In [2]:
create_visualization();

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8700


In [3]:
# indicates how many threads Julia was started with. This is important for the multi-threaded environment
Threads.nthreads()

1

# Create Reinforcement Learning Environment

In [4]:
mutable struct VtolEnv{A,T,ACT,R<:AbstractRNG} <: AbstractEnv # Parametric Constructor for a subtype of AbstractEnv
    action_space::A # action space
    observation_space::Space{Vector{ClosedInterval{T}}} # observation space
    state::Vector{T} # current state space
    action::ACT # action space
    done::Bool # done
    t::T # time
    rng::R # random number generator

    name::String # for multible environments
    visualization::Bool # visualization
    realtime::Bool # realtime

    x_W::Vector{T} # current position
    v_B::Vector{T} # velocity
    R_W::Matrix{T} # current rotation
    ω_B::Vector{T} # rotation velocitiy
    wind_W::Vector{T} # wind
    Δt::T # Δ time

    ################################ TODO ################################
    # Extend the environment here.
    # Everything you need additionaly in your environment also go in here.
    # E.g. a trajectory

    waypoints::Vector{Vector{T}} # waypoints
    rtol::T # proximity tolerance
    τi::Vector{Function{T}} # N trajectories
    T::T # final time
    ######################################################################
end

LoadError: TypeError: in Type{...} expression, expected UnionAll, got Type{Function}

In [5]:
################################ TODO ################################
# You can initialization global constants here.
# E.g. a fixed point in the beginning of training (for testing/overfitting)

# Define global constants for initial position and rotation
const INITIAL_POSITION = [0.0, 0.0, 0.0]
const INITIAL_ROTATION = Quaternion(1.0, 0.0, 0.0, 0.0)

# Function to initialize the environment with the global constants
function init_environment(env::VtolEnv)
    env.x_W = INITIAL_POSITION
    env.R_W = INITIAL_ROTATION
end

######################################################################

LoadError: UndefVarError: Quaternion not defined

In [9]:
# define a keyword-based constructor for the type declared in the mutable struct typedef. 
# It could also be done with the macro Base.@kwdef.
function VtolEnv(;
    rng = Random.GLOBAL_RNG, # random number generation
    name = "vtol",
    visualization = false,
    realtime = false,
    kwargs... # let the function take an arbitrary number of keyword arguments 
)
    
    T = Float64; # explicit type which is used e.g. in state. Cannot be altered due to the poor matrix defininon.

    action_space = Space(
        ClosedInterval{T}[
            0.0..2.0, # propeller 1
            0.0..2.0, # propeller 2
            ], 
    )
    
    state_space = Space( # Three continuous values in state space.
        ClosedInterval{T}[
            ################################ TODO ################################
            # Implement an observation space. 
            # Here is an example space. You can change it if desired.
            # You have to extend it.
            # Orientate yourself on the observation space from the paper.
            
            typemin(T)..typemax(T), # position along x WORLD coordinates
            typemin(T)..typemax(T), # position along z WORLD coordinates
            
            typemin(T)..typemax(T), # orientation along x WORLD coordinates
            typemin(T)..typemax(T), # orientation along z WORLD coordinates
            
            #suggested as above, but in the paper the rotation matrix is used instad???
            
            typemin(T)..typemax(T), # velocity along x BODY coordinates
            typemin(T)..typemax(T), # velocity along y BODY coordinates
            
            typemin(T)..typemax(T), # rotational velocity along z BODY coordinates

            typemin(T)..typemax(T), # position of target along x BODY coordinates
            typemin(T)..typemax(T), # position of target along y BODY coordinates
            
            typemin(T)..typemax(T), # velocity of wind along x BODY coordinates
            typemin(T)..typemax(T), # velocity of wind along y BODY coordinates

            ######################################################################
            ], 
    )
    
    if visualization #visualizes VTOL
        create_VTOL(name, actuators = true, color_vec=[1.0; 1.0; 0.6; 1.0]);
    end

    environment = VtolEnv(
        action_space, # action space
        state_space, # observation space
        zeros(T, length(state_space)), # current state space                       ??? At least target pos not 0???
        rand(action_space), # initialization action
        false, # episode done 
        0.0, # time
        rng, # random number generator  
        
        name,
        visualization,
        realtime,
        
        zeros(T, 3), # x_W, current position
        zeros(T, 3), # v_B, velocity
        [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], # R_W, current rotation, Float64... so T needs to be Float64
        zeros(T, 3), # ω_B
        zeros(T, 3), # wind_W
        T(0.025), # Δt 
        
        [10.0, 10.0] # W, target position in BODY coordinates             Probably randomize, guaranteed overfitting without
        ################################## TODO ##################################
        # Initialization everything you need additionaly in your environment here.

        generate_trajectory(num_waypoints::Int),
        1e-8, # proximity tolerance
        [], # N trajectories
        0.0, # final time
        
        ##########################################################################
    )
    
    
    reset!(environment)
    
    return environment
    
end;

Just for explanation:

1. A mutable Struct is created. A struct is a constructor and a constructor is a function that creates new objects.
2. A outer keyword-based constructor method is added for the type declared in the mutable struct typedef before.

So now we have a function with two methods. Julia will decide which method to call by multiple dispatch.

In [10]:
methods(VtolEnv)

# 1 method for type constructor:
[1] VtolEnv(; rng, name, visualization, realtime, kwargs...) in Main at In[9]:3

# Define the RL interface

In [11]:
Random.seed!(env::VtolEnv, seed) = Random.seed!(env.rng, seed)
RLBase.action_space(env::VtolEnv) = env.action_space
RLBase.state_space(env::VtolEnv) = env.observation_space
RLBase.is_terminated(env::VtolEnv) = env.done
RLBase.state(env::VtolEnv) = env.state

In [ ]:
function computeReward(env::VtolEnv{A,T}) where {A,T}

    ################################ TODO ################################
    # Implement the reward function.
    # Orientate on the paper.
    
    reward = 0.0
    drone_position = env.x_w;
    gates = env.waypoints;
    l_min_index, phi_l = Utils.calculate_progress(gates, drone_position);



    return reward

    #this rewards makes drone go straight up :)
    stay_alive = 3.0
    not_upright_orientation = abs(env.state[1]-pi*0.5)*10.0
    not_centered_position = abs(env.state[3])*10.0
    hight = env.state[4]*100.0
    return stay_alive - not_upright_orientation - not_centered_position + hight
    ######################################################################
end


RLBase.reward(env::VtolEnv{A,T}) where {A,T} = computeReward(env)

In [ ]:
function RLBase.reset!(env::VtolEnv{A,T}) where {A,T}
    # Visualize initial state
    if env.visualization
        set_transform(env.name, env.x_W, QuatRotation(env.R_W));
        set_actuators(env.name, [0.0; 0.0; 0.0; 0.0]);
    end
    
    env.x_W = [0.0; 0.0; 0.0];
    env.v_B = [0.0; 0.0; 0.0];
    env.R_W = Matrix(UnitQuaternion(RotZ(-pi/2.0)*RotY(-pi/2.0)*RotX(pi)));

    env.ω_B = [0.0; 0.0; 0.0];
    env.wind_W = [0.0; 0.0; 0.0];
    
    env.t = 0.0;
    env.action = [0.0, 0.0];
    env.done = false;
    
    # env.W = [?; ?; ?] 

    ################################ TODO ################################
    # Reset environment. 
    # Is called if the training terminates 
    # (e.g. if drone crashes or successfully reaches point)
    # HINT: Everything you added to your environment needs to be reseted.
    #       Compare it with the initialization.
    init_environment(env);
    
    
    ######################################################################
    
    nothing

end;

In [ ]:
# defines a methods for a callable object.
# So when a VtolEnv object is created, it has this method that can be called
function (env::VtolEnv)(a)
    # set the propeller trust and the two flaps 2D case
    # flaps set to 0.0
    next_action = [a[1], a[2], 0.0, 0.0]
   
    _step!(env, next_action)
end

In [ ]:
env = VtolEnv();

In [ ]:
methods(env) # Just to explain which methods the object has

In [ ]:
function _step!(env::VtolEnv, next_action)
    # caluclate wind impact
    v_in_wind_B = vtol_add_wind(env.v_B, env.R_W, env.wind_W);
    
    # caluclate aerodynamic forces
    torque_B, force_B = vtol_model(v_in_wind_B, next_action, eth_vtol_param);
    
    # Limit to 2D
    force_B[3] = 0.0; # Body Z
    env.v_B[3] = 0.0;
    torque_B[1] = 0.0; torque_B[2] = 0.0;  # Body X and Y
    env.ω_B[1] = 0.0; env.ω_B[2] = 0.0;
    
    # integrate rigid body dynamics for Δt                      TO DO: add calculation of W
    env.x_W, env.v_B, env.R_W, env.ω_B, time = rigid_body_simple(torque_B, force_B, env.x_W, env.v_B, env.R_W, env.ω_B, env.t, env.Δt, eth_vtol_param);
    
    if env.realtime
        sleep(env.Δt); # just a dirty hack. this is of course slower than real time.
    end

    # Visualize the new state 
    if env.visualization
        set_transform(env.name, env.x_W, QuatRotation(env.R_W));
        set_actuators(env.name, next_action);
    end 

    env.t += env.Δt
    
    env.state[1] = env.x_W[1]; # position along x
    env.state[2] = env.x_W[3]; # position along z
    
    env.state[3] = env.R_W[1,1]; # orientation along x
    env.state[4] = env.R_W[3,1]; # orientation along z
    
    env.state[5] = env.v_B[1]; # velocity along x BODY coordinates
    env.state[6] = env.v_B[2]; # velocity along y BODY coordinates
    
    env.state[7] = env.ω_B[3];  # rotational velocity along z BODY coordinates

    ################################ TODO ################################
    # Implement step.
    # HINT: This is relatet to your environment. 
    #       Compare to struct VtolEnv.
    #       How does it change in every step.
    
    ######################################################################
        
    
    
    ################################ TODO ################################
    # Add termination criterias.
    # Use many termination criteria so that you do not train unnecessarily in wrong areas.
    env.done = 
        norm(env.ω_B) > 100.0 || # stop if body rate is too high
        norm(env.v_B) > 100.0 || # stop if body is too fast
    
    ######################################################################

    nothing
end;

changed to 10s (5s before) per point and 5.0m too far off path (2.0 before)

In [ ]:
RLBase.test_runnable!(env)

Show an overview of the environment.

# Setup of a reinforcement learning experiment.

In [ ]:
seed = 123    
rng = StableRNG(seed)
    N_ENV = 8
    UPDATE_FREQ = 1024
    
    
    # define multiple environments for parallel training
    env = MultiThreadEnv([
        # use different names for the visualization
        VtolEnv(; rng = StableRNG(hash(seed+i)), name = "vtol$i") for i in 1:N_ENV
    ])

In [ ]:
# Define the function approximator
# (optional) TODO: change architecture
# TODO: research briefly what Actor Critic is
# (optional) TODO: change optimizer
# TODO: research what ADAM is
    ns, na = length(state(env[1])), length(action_space(env[1]))
    approximator = ActorCritic(
                actor = GaussianNetwork(
                    pre = Chain(
                    Dense(ns, 16, tanh; initW = glorot_uniform(rng)),#
                    Dense(16, 16, tanh; initW = glorot_uniform(rng)),
                    ),
                    μ = Chain(Dense(16, na; initW = glorot_uniform(rng))),
                    logσ = Chain(Dense(16, na; initW = glorot_uniform(rng))),
                ),
                critic = Chain(
                    Dense(ns, 16, tanh; initW = glorot_uniform(rng)),
                    Dense(16, 16, tanh; initW = glorot_uniform(rng)),
                    Dense(16, 1; initW = glorot_uniform(rng)),
                ),
                optimizer = ADAM(1e-3),
            );

In [ ]:
    agent = Agent( # A wrapper of an AbstractPolicy
        # AbstractPolicy: the policy to use
        # (optional) TODO: change eventually
        # TODO: research briefly what PPO is
        policy = PPOPolicy(;
                    approximator = approximator |> gpu,
                    update_freq=UPDATE_FREQ,
                    dist = Normal,
                    # For parameters visit the docu: https://juliareinforcementlearning.org/docs/rlzoo/#ReinforcementLearningZoo.PPOPolicy
                    ),
        
        # AbstractTrajectory: used to store transitions between an agent and an environment source
        trajectory = PPOTrajectory(;
            capacity = UPDATE_FREQ,
            state = Matrix{Float64} => (ns, N_ENV),
            action = Matrix{Float64} => (na, N_ENV),
            action_log_prob = Vector{Float64} => (N_ENV,),
            reward = Vector{Float64} => (N_ENV,),
            terminal = Vector{Bool} => (N_ENV,),
        ),
    );


In [ ]:
function saveModel(t, agent, env)
    model = cpu(agent.policy.approximator)   
    f = joinpath("./RL_models/", "vtol_2D_ppo_$t.bson") # TODO: save model here
    @save f model
    println("parameters at step $t saved to $f")
end;

In [ ]:
function validate_policy(t, agent, env)
    run(agent.policy, test_env, StopAfterEpisode(1), episode_test_reward_hook)
    # the result of the hook
    println("test reward at step $t: $(episode_test_reward_hook.rewards[end])")
    
end;

episode_test_reward_hook = TotalRewardPerEpisode(;is_display_on_exit=false)
# create a env only for reward test
test_env = VtolEnv(;name = "testVTOL", visualization = true, realtime = true);

In [ ]:
# agent.policy.approximator = loadModel(); # TODO: un/comment

In [ ]:
ReinforcementLearning.run(
    agent,
    env,
    StopAfterStep(1_500_000),
    ComposedHook(
        DoEveryNStep(saveModel, n=100_000), 
        DoEveryNStep(validate_policy, n=10_000)),
)

In [ ]:
plot(episode_test_reward_hook.rewards)

In [ ]:
close_visualization(); # closes the MeshCat visualization